In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
file = '/workspace/logs/nvprof_resnext101_32x48d_b2.csv'
df = pd.read_csv(file, skiprows=5,
                 names=['start','duration','gridX','gridY','gridZ','blockX','blockY','blockZ',
                        'registersPerThread','staticSMem','dynamicSMem','size','throughput',
                        'srcMemType','dstMemType','device','context','stream','name','corrid'])
# staticSMem - KB, dynamicSMem - KB, size - MB, throughput - GB/s
print(df.shape)
df.drop(['gridX','gridY','gridZ','blockX','blockY','blockZ','srcMemType','dstMemType','device'], axis=1, inplace=True)
df.dropna(subset=['registersPerThread','staticSMem','dynamicSMem','size','throughput'], how='all', inplace=True)

# demangling the name
df['name'] = df['name'].apply(torch._C._demangle)
print(df.shape)
print(df.head())

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(590223, 20)
(39674, 11)
           start  duration  registersPerThread  staticSMem  dynamicSMem  \
237  9426588.045     6.656                 NaN         NaN          NaN   
247  9426818.124     1.664                 NaN         NaN          NaN   
257  9426876.747     1.664                 NaN         NaN          NaN   
267  9426930.699     1.632                 NaN         NaN          NaN   
277  9426974.315     1.664                 NaN         NaN          NaN   

         size  throughput  context  stream                name  corrid  
237  0.035889    5.265555      1.0     7.0  [CUDA memcpy HtoD]     236  
247  0.000244    0.143280      1.0     7.0  [CUDA memcpy HtoD]     245  
257  0.000244    0.143280      1.0     7.0  [CUDA memcpy HtoD]     254  
267  0.000244    0.146090      1.0     7.0  [CUDA memcpy HtoD]     263  
277  0.000244    0.143280      1.0     7.0  [CUDA memcpy HtoD]     272  


In [3]:
print(df.dtypes)
# print(df.tail(50))

start                 float64
duration              float64
registersPerThread    float64
staticSMem            float64
dynamicSMem           float64
size                  float64
throughput            float64
context               float64
stream                float64
name                   object
corrid                  int64
dtype: object


In [4]:
# print(df[(df['size'] > 2.0) & (df['size'] < 2.5) & (df['name'] == '[CUDA memcpy HtoD]')])
batchStartIndices = df.index[(df['size'] > 2.0) & (df['size'] < 2.5) & (df['name'] == '[CUDA memcpy HtoD]')].tolist()
display(batchStartIndices)

# print(df[df['name'].str.contains("gatherTopK")])
accComputeStartIndices = df.index[df['name'].str.contains("gatherTopK")].tolist()
display(accComputeStartIndices)

[6692, 92025, 182627, 273166, 363605]

[91413, 182286, 272825, 363264, 453703]

In [5]:
start = batchStartIndices[1]
end = accComputeStartIndices[1] - 1
b2df = df.loc[start:end,:]

b2df = b2df[~((b2df['name'] == '[CUDA memset]') | (b2df['name'] == '[CUDA memcpy DtoD]') | (b2df['name'] =='[CUDA memcpy DtoH]'))]
print(b2df.shape)
display(b2df.head())
display(b2df.tail())

(6465, 11)


,start,duration,registersPerThread,staticSMem,dynamicSMem,size,throughput,context,stream,name,corrid
92025,1.089080e+07,311.326,NaN,NaN,NaN,2.343750,7.351838,1.0,7.0,[CUDA memcpy HtoD],84083
92054,1.089151e+07,1.664,NaN,NaN,NaN,0.000015,0.008955,1.0,7.0,[CUDA memcpy HtoD],84108
92104,1.089177e+07,2.464,16.0,0.0,0.0,NaN,NaN,1.0,7.0,cudnn::gemm::computeOffsetsKernel(cudnn::gemm:...,84158
92110,1.089180e+07,118.784,128.0,16.0,0.0,NaN,NaN,1.0,7.0,volta_scudnn_128x64_relu_medium_nn_v1,84161
92141,1.089194e+07,3.008,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<128, 4, at...",84189


,start,duration,registersPerThread,staticSMem,dynamicSMem,size,throughput,context,stream,name,corrid
182281,1.171870e+07,32.224,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at...",166115
182282,1.171874e+07,31.615,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at...",166128
182283,1.171878e+07,2.176,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at...",166141
182284,1.171879e+07,1.984,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at...",166154
182285,1.171880e+07,1.920,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at...",166167


In [6]:
pd.options.display.max_colwidth = 200

In [21]:
# input df should be a dataframe containing entries corresponding to a batch
def isNativeRow(name):
    """
    Returns True if a pytorch native operation is performed based on string matching with name attribute.
    """
    return "at::native::" in name

def isfixIndexJoinID(index):
    """Return True if index in list. We want to increment joinID by 1 after these to sync with pytorch trace"""
    indices = [93566,97472,102542,122429]
    return index in indices

def addJoinIDs(df):
    df['joinID'] = np.nan
    joinID_loc = df.columns.get_loc('joinID')
    joinid = 0
    
    assert df.iloc[0, df.columns.get_loc('name')] == '[CUDA memcpy HtoD]', "input df doesn't match expected format"
    df.iloc[0, joinID_loc] = joinid
    joinid += 1
    
    assert df.iloc[1, df.columns.get_loc('name')] == '[CUDA memcpy HtoD]', "input df doesn't match expected format"
    df.iloc[1, joinID_loc] = joinid
    joinid += 1
    isContinue = False
#     display(df.head(20))
    for index, row in df.iloc[2:].iterrows():
        # do something
        if isContinue and isNativeRow(row['name']):
            joinid += 1
            isContinue = False
            
        df.loc[index, 'joinID'] = joinid
        
        if isNativeRow(row['name']) or isfixIndexJoinID(index):
            joinid += 1
        elif not isContinue:
            isContinue = True
        
    df['joinID'] = df['joinID'].astype(int)

addJoinIDs(b2df)
display(b2df.tail(50))

,start,duration,registersPerThread,staticSMem,dynamicSMem,size,throughput,context,stream,name,corrid,joinID
182236,1.171244e+07,615.196,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165530,2003
182237,1.171306e+07,613.117,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165543,2004
182238,1.171368e+07,2.144,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::mul_kernel_cuda(at::TensorIterator&)::{lambda()#1}::operator()() const::{lam...",165556,2005
182239,1.171369e+07,2.144,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165569,2006
182240,1.171370e+07,1.888,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165582,2007
182241,1.171371e+07,2.080,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::mul_kernel_cuda(at::TensorIterator&)::{lambda()#1}::operator()() const::{lam...",165595,2008
182242,1.171372e+07,2.144,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165608,2009
182243,1.171373e+07,2.016,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165621,2010
182244,1.171374e+07,255.998,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::mul_kernel_cuda(at::TensorIterator&)::{lambda()#1}::operator()() const::{lam...",165634,2011
182245,1.171400e+07,365.726,16.0,0.0,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()() const::{lambda()#4}::operator()() ...",165647,2012


In [17]:
b2df['joinID'] = b2df['joinID'].astype(int)
display(b2df.head(50))
b2df.to_csv('/workspace/logs/tmp_b2df.csv')

,start,duration,registersPerThread,staticSMem,dynamicSMem,size,throughput,context,stream,name,corrid,joinID
92025,1.089080e+07,311.326,NaN,NaN,NaN,2.343750,7.351838,1.0,7.0,[CUDA memcpy HtoD],84083,0
92054,1.089151e+07,1.664,NaN,NaN,NaN,0.000015,0.008955,1.0,7.0,[CUDA memcpy HtoD],84108,1
92104,1.089177e+07,2.464,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,cudnn::gemm::computeOffsetsKernel(cudnn::gemm::ComputeOffsetsParams),84158,2
92110,1.089180e+07,118.784,128.0,16.000000,0.0,NaN,NaN,1.0,7.0,volta_scudnn_128x64_relu_medium_nn_v1,84161,2
92141,1.089194e+07,3.008,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<128, 4, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()(...",84189,3
92229,1.089224e+07,232.127,32.0,0.140625,0.0,NaN,NaN,1.0,7.0,"void cudnn::detail::bn_fw_tr_1C11_kernel_NCHW<float, float, int=512, bool=1, int=1>(cudnnTensorStruct, float const *, cudnn::detail::bn_fw_tr_1C11_kernel_NCHW<float, float, int=512, bool=1, int=1>...",84279,4
92291,1.089248e+07,32.863,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::threshold_kernel_impl<float>(at::TensorIterator&, float, float)::{lambda(float, float)#1}>(at::TensorIterator&, ...",84303,5
92301,1.089253e+07,33.632,24.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::_GLOBAL__N__63_tmpxft_00008bdb_00000000_11_DilatedMaxPool2d_compute_75_cpp1_ii_db999de0::MaxPoolForward<float, float>(int, float const *, int, int, int, int, int, int, int, int, i...",84341,6
92350,1.089303e+07,1.920,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,cudnn::gemm::computeOffsetsKernel(cudnn::gemm::ComputeOffsetsParams),84397,7
92355,1.089305e+07,282.975,128.0,16.000000,0.0,NaN,NaN,1.0,7.0,volta_scudnn_128x64_relu_interior_nn_v1,84400,7


In [10]:
len(b2df.index[b2df['name'].str.contains("at::native::")].tolist())

1633

In [25]:
display(b2df[(b2df['joinID'] > 410) & (b2df['joinID'] <420)])

,start,duration,registersPerThread,staticSMem,dynamicSMem,size,throughput,context,stream,name,corrid,joinID
122257,1.109490e+07,2.400,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<128, 4, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()(...",107740,411
122278,1.109496e+07,204.607,40.0,0.140625,2.0,NaN,NaN,1.0,7.0,"void cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, int=2, int=0>(cudnnTensorStruct, float const *, cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, i...",107830,412
122350,1.109518e+07,25.504,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<512, 1, at::native::gpu_kernel_impl<at::native::threshold_kernel_impl<float>(at::TensorIterator&, float, float)::{lambda(float, float)#1}>(at::TensorIterator&, ...",107854,413
122372,1.109524e+07,2214.228,63.0,2.250000,0.0,NaN,NaN,1.0,7.0,"void cudnn::detail::implicit_convolve_sgemm<float, float, int=1024, int=5, int=5, int=3, int=3, int=3, int=1, bool=1, bool=0, bool=1>(int, int, int, float const *, int, float*, cudnn::detail::impl...",107905,414
122412,1.109747e+07,2.368,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<128, 4, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()(...",107934,415
122429,1.109753e+07,36.127,40.0,0.140625,2.0,NaN,NaN,1.0,7.0,"void cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, int=2, int=0>(cudnnTensorStruct, float const *, cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, i...",108024,416
122458,1.109761e+07,197.823,63.0,2.250000,0.0,NaN,NaN,1.0,7.0,"void cudnn::detail::implicit_convolve_sgemm<float, float, int=1024, int=5, int=5, int=3, int=3, int=3, int=1, bool=1, bool=0, bool=1>(int, int, int, float const *, int, float*, cudnn::detail::impl...",108084,417
122541,1.109783e+07,2.368,16.0,0.000000,0.0,NaN,NaN,1.0,7.0,"void at::native::elementwise_kernel<128, 4, at::native::gpu_kernel_impl<at::native::gpu_kernel_with_scalars<at::native::add_kernel_cuda(at::TensorIterator&, c10::Scalar)::{lambda()#1}::operator()(...",108113,418
122562,1.109789e+07,35.583,40.0,0.140625,2.0,NaN,NaN,1.0,7.0,"void cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, int=2, int=0>(cudnnTensorStruct, float const *, cudnn::detail::bn_fw_tr_1C11_singleread<float, int=512, bool=1, int=1, i...",108203,419
